In [1]:
import torch
from operator import itemgetter
from torch.utils.data import DataLoader
import random
import numpy as np
import math
import os
from update_utilities import update_utilities_class
import pickle

In [2]:
with open('saved_encoder_dict.pkl','rb') as f:
    encode_char = pickle.load(f)

with open('saved_decoder_dict.pkl','rb') as f:
    decode_char = pickle.load(f)

print(len(encode_char))
vocab_size = len(encode_char)

74


In [3]:
encode = lambda string: [encode_char[s] for s in string]
decode = lambda nums: ''.join([decode_char[n] for n in nums])

In [4]:
import Transformer
block_size=512
device='cpu'
transformer = Transformer.TransformerClass(vocab_size=vocab_size,emb_dim=512,n_layer=8,num_heads=8,block_size=block_size,
                               dropout_rate_attention=0.1,dropout_rate_ff=0.2,dropout_rate_pos_enc=0.1, 
                               is_decoder = True, ff_multiplier = 4).to(device)

In [5]:
print(round(sum(p.numel() for p in transformer.parameters())/1e6,2), 'M parameters')

25.28 M parameters


In [6]:
transformer.load_state_dict(torch.load("base_line_GPT stats/base_line_GPT weights/base_line_GPT_last.pth"))

<All keys matched successfully>

In [10]:
def generate(model,max_new_tokens,encoder,decoder,block_size=512):
    print(f"Please enter some text to start the text generation, ideally the length you type is between 20 and 50 words, model will generate {max_new_tokens} characters\n")
    user_input = input(f"Enter here: ")
    input_encode = torch.tensor(encoder(user_input))
    output = list(input_encode)
    if len(input_encode) < block_size:
        pad_length = block_size - len(input_encode)
        idx = torch.zeros((1,block_size),dtype=torch.long)
        idx[:,pad_length:] = input_encode
        #print(decoder(idx.squeeze().tolist()))
    else:
        idx = input_encode[-block_size:].unsqueeze(0)
        #print(decoder(idx.squeeze().tolist()))
    
    print_status = False
    print_index_start = len(output)
    print("\n\n")
    print("Generating..........................\n")
    sentence_end = encoder(".!?' ")
    

    

    for _ in range(max_new_tokens):
        if len(output) % 100 == 0:
            print_status = True
        if print_status and output[-1] in sentence_end:
            print_setence = decoder(output[print_index_start:])
            print_index_start = len(output)
            print_status=False
            print(print_setence + "\n")
        idx_cond = idx[:,-block_size:]
        logits = model(idx_cond) # B, T, vocab_size
        logits = logits[:,-1,:] # logits for the last character
        probs = torch.nn.functional.softmax(logits.squeeze(),dim=0)
        idx_next = torch.multinomial(probs,num_samples=1)
        output.append(idx_next.squeeze().item())
        idx = torch.cat((idx,idx_next.unsqueeze(0)),dim=1)
    
    
    


In [14]:
generate(transformer,2000,encoder = encode, decoder = decode,block_size=512)

Please enter some text to start the text generation, ideally the length you type is between 20 and 50 words, model will generate 2000 characters



Enter here:  Legolas and Aragorn sitting in a tree. Kissing. Legolas looks sad that Aragorn has not yet left Arwen. 





Generating..........................

They passed their hands and looked out of silence towards them. The precious things had begun in 

the days of Anar they had been written by some gifty leaves. As their eyes grew the truth. Luthien, 

feared that now clear was the darkness of evening summer, and the fall of Turin as it seemed the oathbreaker 

to were beneath him. By the right they could all be, and Turin knew their weapons died in Nan 

Elmoth, and there many of the Trees being woke in a stride, and there fell Curufin gate fell upon 

his head, and on the saddle of Turin, and treachery did his tatter in torment; and his eyes shining 

in the body and skill the blades of Doriath. By the starlit mere was gleamed with two flowers of manhood, 

and King Thingol gave there to the sons of Finrod; but still Beleg watched ever and came over 

the mountains There in the night there mantled all the houses of Thingol, and drew forth even to Doriath, 

seeking vessels for peril. For the Na

In [21]:
a = torch.randint(70, (100,))
pad_length = 512 - len(a)
idx = torch.zeros((1,block_size),dtype=torch.long)
idx[:,pad_length:] = a
print(idx.shape)
b = transformer(idx)
print(b.shape)

torch.Size([1, 512])
torch.Size([1, 512, 74])


In [31]:
logits = b[:,-1,:].squeeze()
logits.shape

torch.Size([74])

In [32]:
logits

tensor([ 5.6646,  4.8699,  0.5909, -1.1043,  4.5587,  4.2549,  2.8798,  6.3165,
         6.9580,  3.1944,  7.9979,  3.0714,  7.6264,  2.1841,  5.2547,  2.5160,
         5.5916,  5.4049,  3.4680,  0.3700,  2.4109,  3.1376,  2.5324,  0.3565,
         2.3291,  8.1200,  4.0588,  2.1128,  2.5585,  3.4352, -0.9041,  0.8907,
         2.9982,  3.0420,  4.4203,  5.7570,  2.0232,  3.7661,  2.8905,  3.5078,
         4.2205,  0.7041,  3.2672,  1.9475,  1.5920,  5.3364, -0.4227, -0.8958,
        -1.7573, -4.9720, -3.4157,  3.1858, -2.9820, -0.9897, -1.8671, -7.1939,
         4.2730, -3.0023, -5.3467, -1.6149, -4.3354,  1.3004, -1.3381, -2.5372,
        -4.8071, -1.4945, -4.1417,  2.1203, -2.7545,  0.3176, -2.0061, -6.9457,
         2.3136, -1.0229], grad_fn=<SqueezeBackward0>)

In [37]:
probs = torch.nn.functional.softmax(logits,dim=0)
probs

tensor([2.3017e-02, 1.0397e-02, 1.4407e-04, 2.6447e-05, 7.6165e-03, 5.6214e-03,
        1.4212e-03, 4.4175e-02, 8.3902e-02, 1.9466e-03, 2.3734e-01, 1.7212e-03,
        1.6371e-01, 7.0876e-04, 1.5277e-02, 9.8775e-04, 2.1398e-02, 1.7753e-02,
        2.5591e-03, 1.1552e-04, 8.8920e-04, 1.8391e-03, 1.0041e-03, 1.1396e-04,
        8.1939e-04, 2.6818e-01, 4.6201e-03, 6.5998e-04, 1.0306e-03, 2.4766e-03,
        3.2309e-05, 1.9443e-04, 1.5997e-03, 1.6714e-03, 6.6324e-03, 2.5246e-02,
        6.0342e-04, 3.4480e-03, 1.4365e-03, 2.6631e-03, 5.4310e-03, 1.6134e-04,
        2.0935e-03, 5.5940e-04, 3.9205e-04, 1.6578e-02, 5.2288e-05, 3.2576e-05,
        1.3764e-05, 5.5290e-07, 2.6214e-06, 1.9299e-03, 4.0448e-06, 2.9658e-05,
        1.2334e-05, 5.9933e-08, 5.7238e-03, 3.9636e-06, 3.8012e-07, 1.5872e-05,
        1.0450e-06, 2.9290e-04, 2.0932e-05, 6.3107e-06, 6.5200e-07, 1.7902e-05,
        1.2684e-06, 6.6493e-04, 5.0778e-06, 1.0962e-04, 1.0733e-05, 7.6817e-08,
        8.0678e-04, 2.8689e-05], grad_fn

In [87]:

d = torch.multinomial(probs,num_samples=1).unsqueeze(0)
print(d.shape)
print(d)

torch.Size([1, 1])
tensor([[10]])


In [92]:
d.squeeze().item()

10